# **Bow(Haskell): Traducido PySpark**
```
NOMBRE: JORGE ANDRE SALCEDO HURTADO
CURSO: MINERIA DE DATOS
DOCENTE: CARLOS FERNANDO MONTOYA CUBAS
```

#### Implementar los algoritmos del archivo bow.hs en PySpark


# 1. Inicializamos nuestra variable de pyspark

In [1]:
from pyspark import SparkContext
sc =SparkContext()

22/01/09 12:53:54 WARN Utils: Your hostname, MR resolves to a loopback address: 127.0.1.1; using 192.168.1.20 instead (on interface enp9s0)
22/01/09 12:53:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/09 12:53:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/09 12:53:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


# 2. Implementación

## 2.1. Bag of Words
El modelo "bolsa de palabras" es un método que se utiliza en el procesado del lenguaje para representar documentos ignorando el orden de las palabras. En este modelo, cada documento parece una bolsa que contiene algunas palabras.

In [2]:
def bagOfWords(corpus = []):
    """Devuelve el corpus con todas las letras cambiadas a minuscula, quita espacios en blanco y las letras mayores a 2

    Args:
        corpus (List): Lista de textos(corpus).

    Returns:
        str: El corpus procesado.
    """
    # creamos la RDD en base al corpus
    bow = sc.parallelize(corpus)
    # eliminamos las lineas en blanco
    bow = bow.filter(lambda x: len(x)>0)
    # separamos en palabras cada linea
    bow = bow.map(lambda x: x.split(" "))
    
    # realizamos una iteracion por cada palabra dentro de cada linea
    # la funcion lambda interna devuelve las palabras en minuscula
    bow = bow.map(lambda text: list(map((lambda word: word.lower()),text)))
    
    # eliminamos las palabras que tengan una longitud menor a 2
    bow = bow.map(lambda text: list(filter(lambda word: len(word)>2,text)))
    return bow.collect()

In [3]:
Corpus = ["Project Gutenberg’s The Complete Works of William Shakespeare, by William",
            "Shakespeare",
            "This eBook is for the use of anyone anywhere in the United States and",
            "most other parts of the world at no cost and with almost no restrictions",
            "whatsoever.  You may copy it, give it away or re-use it under the terms",
            "of the Project Gutenberg License included with this eBook or online at",
            "www.gutenberg.org.  If you are not located in the United States, you’ll",
            "have to check the laws of the country where you are located before using",
            "this ebook."]
print(bagOfWords(Corpus))         

[['project', 'gutenberg’s', 'the', 'complete', 'works', 'william', 'shakespeare,', 'william'], ['shakespeare'], ['this', 'ebook', 'for', 'the', 'use', 'anyone', 'anywhere', 'the', 'united', 'states', 'and'], ['most', 'other', 'parts', 'the', 'world', 'cost', 'and', 'with', 'almost', 'restrictions'], ['whatsoever.', 'you', 'may', 'copy', 'it,', 'give', 'away', 're-use', 'under', 'the', 'terms'], ['the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'online'], ['www.gutenberg.org.', 'you', 'are', 'not', 'located', 'the', 'united', 'states,', 'you’ll'], ['have', 'check', 'the', 'laws', 'the', 'country', 'where', 'you', 'are', 'located', 'before', 'using'], ['this', 'ebook.']]


## 2.2. ***TF*** (Term Frecuency): Es la frecuencia con la que aparece la palabra en un documento del corpus. Esta se define como:
    
### $$tf(t,d) = 1 + log(f_{t,d})$$

In [4]:
import numpy as np
def tf(corpus = []):
    """Halla el TF(Term Frecuency) de un conjunto de documentos.

    Args:
        corpus (List): Lista de textos(corpus).

    Returns:
        str: Una lista de tuplas compuesto por (palabra, TF).
    """
    # creamos la RDD en base al corpus
    tfRDD = sc.parallelize(corpus)
    # separamos en palabras cada tweet
    tfRDD = tfRDD.map(lambda x: x.split(" "))
    # realizamos una iteracion por cada palabra dentro de cada tweet
    # la funcion lambda interna devuelve en una tupla:
    # 1. La palabra
    # 2. La operacion de TF, para lo cual se halla el conteo de la palabra en el tweet que se 
    # esté analizando ese momento y la longitud del tweet
    tfRDD = tfRDD.map(lambda tweet: tuple(map((lambda word: (word, 1+np.log10(tweet.count(word)/len(tweet)))),tweet))).collect()
    # finalmente debido a que puede haber palabras repetidas para las que se hallo el TF, 
    # con una operacion set, eliminamos estos repetidos
    tfLista = list(map(set,tfRDD))
    # devolvemos una lista, donde cada elemento es otra lista de tuplas que contiene la palabra y su TF
    return tfLista

### Ejecucion

In [5]:
tfValues = tf(Corpus)
# mostramos el resultado
for line in tfValues:
    print(line)

{('of', 0.0), ('Project', 0.0), ('The', 0.0), ('Shakespeare,', 0.0), ('Gutenberg’s', 0.0), ('Complete', 0.0), ('Works', 0.0), ('by', 0.0), ('William', 0.30102999566398125)}
{('Shakespeare', 1.0)}
{('This', -0.14612803567823818), ('eBook', -0.14612803567823818), ('is', -0.14612803567823818), ('States', -0.14612803567823818), ('anywhere', -0.14612803567823818), ('and', -0.14612803567823818), ('United', -0.14612803567823818), ('the', 0.15490195998574308), ('use', -0.14612803567823818), ('for', -0.14612803567823818), ('of', -0.14612803567823818), ('in', -0.14612803567823818), ('anyone', -0.14612803567823818)}
{('parts', -0.14612803567823818), ('the', -0.14612803567823818), ('and', -0.14612803567823818), ('restrictions', -0.14612803567823818), ('other', -0.14612803567823818), ('cost', -0.14612803567823818), ('of', -0.14612803567823818), ('at', -0.14612803567823818), ('world', -0.14612803567823818), ('no', 0.15490195998574308), ('almost', -0.14612803567823818), ('most', -0.14612803567823818)

## 2.3 ***IDF*** (Inverse Document Frequency): La frecuencia inversa del documento nos indica lo común que es una palabra en el corpus.
    
###    $$idf(t,D) = log(1 + \frac{N}{n_t})$$

In [6]:
def idf(corpus = []):
    """Halla el IDF(Inverse Document Frequency) de un conjunto de documentos.

    Args:
        corpus (List): Lista de los textos(corpus).

    Returns:
        str: Una lista de tuplas compuesto por (palabra, IDF).
    """ 
    # obtenemos el numero total de tweets(documentos)
    total = len(corpus)
    # creamos la RDD a partir del corpus
    idfRDD = sc.parallelize(corpus)
    # realizamos para cada tweet la supresion de terminos repetidos, y al usar la funcion flatMap
    # nos devuelve todos las palabras unicas por tweet en un solo conjunto.
    idfRDD = idfRDD.flatMap(lambda x: list(set(x.split(" "))))
    # asignamos el numero 1 a cada palabra y posteriormente sumamos segun su key.
    idfRDD = idfRDD.map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)
    # aplicamos la formula de idf, utilizando el numero total de documentos
    #  y el término anteriormente hallado para cada palabra
    idfRDD = idfRDD.map(lambda x: (x[0], np.log10(1 + total/x[1])))
    # finalmente devolvemos una lista de tuplas conformado por, la palabra y su termino idf
    return list(idfRDD.collect())

### Ejecucion

In [7]:
idfValues = idf(Corpus)
# mostramos el resultado
print(idfValues)

[('other', 1.0), ('', 0.7403626894942439), ('where', 1.0), ('William', 1.0), ('by', 1.0), ('cost', 1.0), ('almost', 1.0), ('you', 0.7403626894942439), ('use', 1.0), ('States', 1.0), ('at', 0.7403626894942439), ('this', 0.7403626894942439), ('online', 1.0), ('copy', 1.0), ('located', 0.7403626894942439), ('If', 1.0), ('in', 0.7403626894942439), ('it,', 1.0), ('the', 0.3979400086720376), ('most', 1.0), ('You', 1.0), ('ebook.', 1.0), ('Project', 0.7403626894942439), ('anywhere', 1.0), ('United', 0.7403626894942439), ('restrictions', 1.0), ('before', 1.0), ('Gutenberg', 1.0), ('States,', 1.0), ('laws', 1.0), ('of', 0.4471580313422192), ('Gutenberg’s', 1.0), ('are', 0.7403626894942439), ('using', 1.0), ('check', 1.0), ('Works', 1.0), ('terms', 1.0), ('you’ll', 1.0), ('anyone', 1.0), ('for', 1.0), ('This', 1.0), ('under', 1.0), ('it', 1.0), ('included', 1.0), ('Shakespeare', 1.0), ('no', 1.0), ('give', 1.0), ('away', 1.0), ('re-use', 1.0), ('may', 1.0), ('whatsoever.', 1.0), ('country', 1.0)

## 2.4 Term Frequency-Inverse Document Frequency (TF-IDF)


* El TF-IDF (Frecuencia de Termino - Frecuencia Inversa de Documento) es una medida numérica que permite expresar como de relevante es una palabra para un documento en una colección de documentos (o corpus).


* Construir la Bolsa de Palabras con TF-IDF en vez de con frecuencias evita dar "importancia" a texto muy largos y con mucha repetición de palabras, frente a textos cortos y con pocas repeticiones de palabras.

* ***TF-IDF*** queda definido como:
### $$tfidf(t,d,D) = tf(t,d) \cdot idf(t,D)$$

In [8]:
def tf_idf(corpus = []):
    """Halla el TF-IDF de un conjunto de documentos.

    Args:
        corpus (List): Lista de los textos(corpus).

    Returns:
        str: Una lista de tuplas compuesto por (palabra, TF-IDF).
    """ 
    # generamos la lista de terminos y sus TF de nuestro corpus
    tfList = tf(corpus)
    # generamos la lista de terminos y sus IDF de nuestro corpus
    idfList = idf(corpus)
    # declaramos un diccionario vacio
    idfDict = {}
    # convertimos nuestra lista de tuplas de terminos IDF para poder llamarlos de mejro manera
    idfDict = dict(idfList)
    
    # creamos una RDD a partir de nuestro conjunto de palabras y sus TF
    tfIdfRDD = sc.parallelize(tfList)
    # recorremos cada palabra por cada Tweet para multiplicar su TF por el IDF correspondiente 
    # y hallar el TFIDF
    tfIdfRDD = tfIdfRDD.map(lambda twiit: tuple(map(lambda word: (word[0],word[1]*idfDict[word[0]]),twiit)))
    # devolvemos la lista de palabras con su respectivo TF-IDF
    return list(tfIdfRDD.collect())

### Ejecucion

In [9]:
tfIDFValues = tf_idf(Corpus)
# mostramos el resultado
for tweet in tfIDFValues:
    print(tweet)

(('by', 0.0), ('Project', 0.0), ('of', 0.0), ('Works', 0.0), ('Shakespeare,', 0.0), ('Complete', 0.0), ('Gutenberg’s', 0.0), ('The', 0.0), ('William', 0.30102999566398125))
(('Shakespeare', 1.0),)
(('States', -0.14612803567823818), ('use', -0.14612803567823818), ('the', 0.06164168730004222), ('of', -0.06534232475778655), ('for', -0.14612803567823818), ('is', -0.14612803567823818), ('in', -0.10818774550525125), ('United', -0.10818774550525125), ('eBook', -0.10818774550525125), ('This', -0.14612803567823818), ('and', -0.10818774550525125), ('anywhere', -0.14612803567823818), ('anyone', -0.14612803567823818))
(('the', -0.05815019178502592), ('most', -0.14612803567823818), ('of', -0.06534232475778655), ('no', 0.15490195998574308), ('world', -0.14612803567823818), ('restrictions', -0.14612803567823818), ('parts', -0.14612803567823818), ('other', -0.14612803567823818), ('cost', -0.14612803567823818), ('almost', -0.14612803567823818), ('and', -0.10818774550525125), ('with', -0.108187745505251